In [1]:
import os
import glob
import tgt
import sys
import pandas as pd
from re import match, sub, findall, finditer


root_dir = '../'
word_level_timing = root_dir+'word_level_timings'
motion_label = root_dir+'motion_labels' 
original_annotation = root_dir + 'transcriptions_annotations'
# word_level     = root_dir + 'word_level'
# video          = root_dir + 'video' 
# audio          = root_dir +'audio' 
# documents      = root_dir + 'documents', 
# metadata       = root_dir+ 'metadata', 
# transcriptions = root_dir+'transcriptions_annotations'

In [2]:
def get_all_textgrid_files(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".TextGrid"):
                filenames.append(os.path.join(root, file))
    return filenames
annotations = get_all_textgrid_files(original_annotation)
print(annotations)      

['../transcriptions_annotations/r19/r19.TextGrid', '../transcriptions_annotations/r18/r18.TextGrid', '../transcriptions_annotations/r13/r13.TextGrid', '../transcriptions_annotations/r9/r9.TextGrid', '../transcriptions_annotations/r10/r10.TextGrid', '../transcriptions_annotations/r1/r1.TextGrid', '../transcriptions_annotations/r4/r4.TextGrid', '../transcriptions_annotations/r15/r15.TextGrid', '../transcriptions_annotations/r7/r7.TextGrid', '../transcriptions_annotations/r5/r5.TextGrid', '../transcriptions_annotations/r17/r17.TextGrid', '../transcriptions_annotations/r12/r12.TextGrid', '../transcriptions_annotations/r2/r2.TextGrid', '../transcriptions_annotations/r3/r3.TextGrid', '../transcriptions_annotations/r6/r6.TextGrid', '../transcriptions_annotations/r14/r14.TextGrid', '../transcriptions_annotations/r16/r16.TextGrid', '../transcriptions_annotations/r11/r11.TextGrid', '../transcriptions_annotations/r8/r8.TextGrid']


In [3]:
def clean_utt(utt, literal=False):
    if not literal:
        #replace variants, partial and misspoken words with standard spelling
        utt = sub("""<[vpm]="(.+?)">.+?</[vpm]>""", lambda m:m.group(1), utt)
        #remove fillers like "{F aehm}" entirely
        utt = sub("""{.*?}""", "", utt)
        
        #TO DO: resolve complex replacements like "(der + der) + die) Katze"
        
    else:
        #remove brackets from fillers, i.e. "{F aehm}" becomes "aehm"
        utt = sub("""{(.*?)}""",lambda m:m.group(1),utt)
    #remove all remaining xml-style tags    
    utt = sub("""<.*?>""","",utt)
    #remove open tags at the end of an utterance (can be removed once problems with the TextGrids are fixed)
    utt = sub("""<.*$""","",utt)
    #remove all remaining punctuation and brackets
    utt = sub("""[\.:;,\(\)\+\$]""","",utt)
    #remove whitespace at the beginning and end of an utterance
    utt = utt.strip()
    #replace any amount of whitespace with a single space
    utt = sub("""\s+"""," ",utt)
    return utt


In [7]:
for index, f in enumerate(annotations):
    clear_text = list()
    textgrid = tgt.read_textgrid(f)
    tier_names = textgrid.get_tier_names()
    print(tier_names)
    for names in tier_names:
        text_tier = textgrid.get_tier_by_name(names)
        for annotation in text_tier.annotations:
            annotation.text = clean_utt(annotation.text)
    tgt.io.write_to_file(textgrid, './clean_text/'+str(f.split('/')[3].split('.')[0])+".textgrid")
    # dataframe = pd.DataFrame(clear_text, columns=['file_name', 'tier_name', 'text', 'clean_text', 'start_time', 'end_time'])
    # # print(f.split('/')[3].split('.')[0])
    # dataframe.to_csv('./clean_text/'+str(f.split('/')[3].split('.')[0])+'.csv')

['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laughter', 'A-utts']
['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laughter', 'A-utts']
['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laughter', 'A-utts']
['B-utts', 'B-turns', 'A-turns', 'Comments', 'A-laughter', 'Part', 'O', 'A-utts', 'B-laughter']
['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laughter', 'A-utts']
['B-utts', 'B-turns', 'A-turns', 'Comments', 'A-laughter', 'Part', 'B-laughter', 'A-utts']
['B-utts', 'B-turns', 'A-turns', 'Comments', 'A-laughter', 'Part', 'B-laughter', 'A-utts']
['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laughter', 'A-utts']
['B-utts', 'B-turns', 'A-turns', 'Comments', 'A-laughter', 'Part', 'O', 'A-utts', 'B-laughter']
['B-utts', 'B-turns', 'A-turns', 'O', 'A-laughter', 'Part', 'Comments', 'A-utts', 'B-laughter']
['B-turns', 'A-turns', 'Comments', 'B-utts', 'Part', 'B-laughter', 'A-laugh